In [7]:
import os
import shutil
import time

from offline_dataset.dataset_creater import GymParallelSampler

from envs.env_creator import ibgym_env_creator, env_creator
from state_quantization.transforms import quantize_transform_creator
from q_learning.algorithm import QLPolicy
from ppo.policy import LSTMPPOPolicy

In [8]:
episodes = 100
steps_per_episode = 1000
workers = 8
#env_kwargs = {'steps_per_episode': steps_per_episode, 'device':'cpu'}
env_kwargs = {'steps_per_episode': steps_per_episode}
writer_path = os.path.join("tmp", "ibqf-out")
#policy_save_path = 'dynamic_programming/ib_policy.pkl'
#policy = DPPolicy.load(policy_save_path)
policy_save_path = 'q_learning/q_policy.pkl'
policy = QLPolicy.load(policy_save_path)
#policy_save_path = 'ppo/lstm_ppo_policy.pkl'
#policy = LSTMPPOPolicy.load(policy_save_path)
q_transform_kwargs = {'device': 'cpu', 'keys': ['obs', 'new_obs'], 'reshape': (steps_per_episode, -1, 6)}

In [9]:
if os.path.exists(writer_path) and os.path.isdir(writer_path):
    shutil.rmtree(writer_path)

In [10]:


start = time.time()
parallel_sampler = GymParallelSampler(env_creator=env_creator, path=writer_path, episodes=episodes,
                                      workers=workers, env_kwargs=env_kwargs, reward_threshold=None,
                                      #buffer_transform=quantize_transform_creator,
                                      #buffer_transform_kwargs=q_transform_kwargs,
                                      policy=policy)

In [11]:
parallel_sampler.sample()
end = time.time()
print(end - start)

Episodes Sampled: 33
Episodes Sampled: 35
Episodes Sampled: 37
Episodes Sampled: 40
Episodes Sampled: 43
Episodes Sampled: 45
Episodes Sampled: 46
Episodes Sampled: 48
Episodes Sampled: 52
Episodes Sampled: 54
Episodes Sampled: 56
Episodes Sampled: 60
<class 'state_quantization.forcasting_quantization_models.ForcastingQuant'>


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Episodes Sampled: 62
Episodes Sampled: 64
Episodes Sampled: 69
Episodes Sampled: 70
<class 'state_quantization.forcasting_quantization_models.ForcastingQuant'>
Episodes Sampled: 73


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Episodes Sampled: 76
Episodes Sampled: 77
Episodes Sampled: 81
<class 'state_quantization.forcasting_quantization_models.ForcastingQuant'>


...

Episodes Sampled: 94
Episodes Sampled: 95
Episodes Sampled: 96
Episodes Sampled: 97


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


<class 'state_quantization.forcasting_quantization_models.ForcastingQuant'>
Episodes Sampled: 98
Episodes Sampled: 99
Episodes Sampled: 99


/home/hamza/PycharmProjects/StateCompression/venv/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


<class 'state_quantization.forcasting_quantization_models.ForcastingQuant'>
Episodes Sampled: 100
Episodes Sampled: 100
Sampling Finished
62.26485204696655


In [12]:
save_path = os.path.join("tmp", "ibqf-out/ibq_samples.npy")
parallel_sampler.create_merged_dataset(save_path=save_path)

tmp/ibqf-out/ibq_samples.npy


{'obs': array([ 29866.,  29866., 554154., ..., 226986., 226986., 226474.]),
 'actions': array([11, 11, 15, ..., 14, 14,  6]),
 'rewards': array([-205.92054749, -223.18812561, -210.1879425 , ..., -158.1499939 ,
        -194.69735718, -154.59277344]),
 'dones': array([False, False, False, ..., False, False,  True]),
 'new_obs': array([ 29866., 554154., 554154., ..., 226986., 226474.,  95914.]),
 'unroll_id': array([ 0,  0,  0, ..., 12, 12, 12])}